In [514]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.graph_objs import *
import plotly.graph_objs as go
from datetime import datetime
import datetime as dt

rec = pd.read_csv('REC1.csv')

rec=rec.iloc[:19]
rec=rec.iloc[-5:]
rec

,Year,Load,Biomass < 25 MW,Biomass > 25 MW,Geothermal,Existing Hydroelectric,New Hydroelectric,Existing Methane Gas,New Methane Gas,Solar Photovoltaic,...,Class IV P,Class I Non-Thermal S,Class I Thermal S,Class II S,Class III S,Class IV S,Class I Price,Class II Price,Class III Price,Class IV Price
14,2022.0,11155551.0,500000.0,160000.0,70000.0,160000.0,95000.0,240000.0,30000.0,423000.0,...,0.015,0.10,0.02,0.01,0.08,0.015,37.25,34.00,37.50,27.75
15,2023.0,11166783.0,520000.0,200000.0,79000.0,159000.0,90000.0,230000.0,33000.0,450000.0,...,0.015,0.11,0.02,0.01,0.08,0.015,38.13,34.50,37.25,28.00
16,2024.0,11169501.0,500000.0,250000.0,82000.0,162000.0,100000.0,260000.0,35000.0,472000.0,...,0.015,0.12,0.02,0.01,0.08,0.015,36.50,34.10,36.50,28.50
17,2025.0,11151265.0,490000.0,250000.0,90000.0,167000.0,110000.0,210000.0,40000.0,500000.0,...,0.015,0.13,0.02,0.01,0.08,0.015,36.00,33.75,36.00,28.50
18,2026.0,11160479.0,480000.0,240000.0,98000.0,161000.0,105000.0,280000.0,38000.0,550000.0,...,0.015,0.13,0.02,0.01,0.08,0.015,34.00,33.00,34.00,28.50


In [515]:
rec_d_p=[]
rec_d_p = pd.DataFrame(rec_d_p)
rec_d_p['Year'] = rec['Year']

rec_d_p['I_D_P'] = rec['Class I P'].astype(float) * (rec['Load'].astype(float))
rec_d_p['II_D_P'] = rec['Class II P'].astype(float) * (rec['Load'].astype(float))
rec_d_p['III_D_P'] = rec['Class III P'].astype(float) * (rec['Load'].astype(float))
rec_d_p['IV_D_P'] = rec['Class IV P'].astype(float) * (rec['Load'].astype(float))

wind = []
wind = pd.DataFrame(wind)
wind['0'] =rec['Wind'].astype(float)
wind = pd.DataFrame(wind)

biomass = []
biomass = pd.DataFrame(biomass)
biomass['0'] =rec['Biomass > 25 MW'].astype(float)+rec['Biomass < 25 MW'].astype(float)
biomass = pd.DataFrame(biomass)

geothermal = []
geothermal = pd.DataFrame(geothermal)
geothermal['0'] =rec['Geothermal'].astype(float)
geothermal = pd.DataFrame(geothermal)

thermal = []
thermal = pd.DataFrame(thermal)
thermal['0'] =rec['Solar Thermal'].astype(float)
thermal = pd.DataFrame(thermal)

methane = []
methane = pd.DataFrame(methane)
methane['0'] = np.where(rec['Year'] >= 2013, rec['New Methane Gas']+rec['Existing Methane Gas'],0)
methane = pd.DataFrame(methane)

class_II = []
class_II = (rec['Solar Photovoltaic'].astype(float)-((rec['Solar Photovoltaic'].astype(float)*rec['Class II Net-Metering Reduction'].astype(float))))-rec_d_p['II_D_P']
class_II = np.where(class_II > 0, class_II.values, 0)
class_II = pd.DataFrame(class_II)

new_class_III = []
new_class_III = pd.DataFrame(new_class_III)
new_class_III['0'] = rec['New Methane Gas']

new_class_IV = []
new_class_IV = pd.DataFrame(new_class_IV)
new_class_IV['0'] = rec['New Hydroelectric']

In [516]:
rec_d_p['I_S_P'] = wind['0'].values + methane['0'].values + class_II[0].values + new_class_III['0'].values + new_class_IV['0'].values
rec_d_p['II_S_P'] = (rec['Solar Photovoltaic'].astype(float)-((rec['Solar Photovoltaic'].astype(float)*rec['Class II Net-Metering Reduction'].astype(float))))
rec_d_p['III_S_P'] =  (rec['Existing Methane Gas'].astype(float)+rec['New Methane Gas'].astype(float))
rec_d_p['IV_S_P'] = (rec['Existing Hydroelectric'].astype(float)+rec['New Hydroelectric'].astype(float))

In [517]:
rec_d_p['I_SD_P'] = rec_d_p['I_S_P']/rec_d_p['I_D_P']
rec_d_p['II_SD_P'] = rec_d_p['II_S_P']/rec_d_p['II_D_P']
rec_d_p['III_SD_P'] = rec_d_p['III_S_P']/rec_d_p['III_D_P']
rec_d_p['IV_SD_P'] = rec_d_p['IV_S_P']/rec_d_p['IV_D_P']
rec_d_p = rec_d_p.iloc[-5:]

In [518]:
rec_s=[]
rec_s = pd.DataFrame(rec_s)
rec_s['Year'] = rec['Year']
rec_s['I_NT_D'] = rec['Class I Non-Thermal S'].astype(float) * (rec['Load'].astype(float))
rec_s['I_T_D'] = rec['Class I Thermal S'].astype(float) * (rec['Load'].astype(float))
rec_s['II_D'] = rec['Class II S'].astype(float) * (rec['Load'].astype(float))
rec_s['III_D'] = rec['Class III S'].astype(float) * (rec['Load'].astype(float))
rec_s['IV_D'] = rec['Class IV S'].astype(float) * (rec['Load'].astype(float))
rec_s['I_NT_D']


14    1115555.10
15    1228346.13
16    1340340.12
17    1449664.45
18    1450862.27
Name: I_NT_D, dtype: float64

In [519]:
rec_s['I_S'] =wind['0'].values + methane['0'].values + class_II[0].values + new_class_III['0'].values + new_class_IV['0'].values+\
     thermal['0'].values+geothermal['0'].values+biomass['0'].values
rec_s['II_S'] = ((rec['Solar Photovoltaic'].astype(float)-((rec['Solar Photovoltaic'].astype(float)*rec['Class II Net-Metering Reduction'].astype(float)))) +rec['Solar Thermal'].astype(float))
rec_s['III_S'] =  (rec['Existing Methane Gas'].astype(float)+rec['New Methane Gas'].astype(float))
rec_s['IV_S'] = (rec['Existing Hydroelectric'].astype(float)+rec['New Hydroelectric'].astype(float))
rec_s['I_S']


14    2422523.251
15    2545361.251
16    2701248.553
17    2769170.205
18    2974904.963
Name: I_S, dtype: float64

In [520]:

rec_s['I_SD'] = rec_s['I_S']/rec_s['I_NT_D']
rec_s['II_SD'] = rec_s['II_S']/rec_s['II_D']
rec_s['III_SD'] = rec_s['III_S']/rec_s['III_D']
rec_s['IV_SD'] = rec_s['IV_S']/rec_s['IV_D']
rec_s = rec_s.iloc[-5:]

total=pd.merge(rec_d_p, rec_s, on='Year')

total

,Year,I_D_P,II_D_P,III_D_P,IV_D_P,I_S_P,II_S_P,III_S_P,IV_S_P,I_SD_P,...,III_D,IV_D,I_S,II_S,III_S,IV_S,I_SD,II_SD,III_SD,IV_SD
0,2022.0,669333.06,33466.653,390444.285,167333.265,1552523.251,420989.904,270000.0,255000.0,2.319508,...,892444.08,167333.265,2422523.251,560989.904,270000.0,255000.0,2.171585,5.028796,0.302540,1.523905
1,2023.0,670006.98,33500.349,390837.405,167501.745,1600361.251,447861.600,263000.0,249000.0,2.388574,...,893342.64,167501.745,2545361.251,593861.600,263000.0,249000.0,2.072186,5.318108,0.294400,1.486552
2,2024.0,670170.06,33508.503,390932.535,167542.515,1716248.553,469757.056,295000.0,262000.0,2.560915,...,893560.08,167542.515,2701248.553,622757.056,295000.0,262000.0,2.015346,5.575514,0.330140,1.563782
3,2025.0,669075.90,33453.795,390294.275,167268.975,1774170.205,497624.000,250000.0,277000.0,2.651673,...,892101.20,167268.975,2769170.205,662624.000,250000.0,277000.0,1.910215,5.942142,0.280237,1.656015
4,2026.0,669628.74,33481.437,390616.765,167407.185,1984904.963,547386.400,318000.0,266000.0,2.964187,...,892838.32,167407.185,2974904.963,719386.400,318000.0,266000.0,2.050439,6.445838,0.356168,1.588940


In [521]:
supply_fig = go.Figure()
supply_fig.add_trace(
    go.Bar(name='Class I Supply',
        x= total['Year'],
        y= (total['I_S']), 
                marker_color='green'
    ),
)
supply_fig.add_trace(
    go.Bar(name='Class II Supply',
        x= total['Year'],
        y= (total['II_S']), 
                marker_color='grey'
    ),
)
supply_fig.add_trace(
    go.Bar(name='Class III Supply',
        x= total['Year'],
        y= (total['III_S']), 
                marker_color='darkgrey'
    ),
)
supply_fig.add_trace(
    go.Bar(name='Clas IV Supply',
        x= total['Year'],
        y= (total['IV_S']), 
                marker_color='lightgrey'
    ),
)

supply_fig.update_layout(title_text="Standard Supply")
supply_fig.show()


In [522]:
demand_fig = go.Figure()
demand_fig.add_trace(
    go.Bar(name='Class I Demand',
        x= total['Year'],
        y= (total['I_NT_D']), 
        marker_color='Green'
    ),
)
demand_fig.add_trace(
    go.Bar(name='Class II Demand',
        x= total['Year'],
        y= (total['II_D']), 
        marker_color='grey'
    ),
)
demand_fig.add_trace(
    go.Bar(name='Class III Demand',
        x= total['Year'],
        y= (total['III_D']), 
        marker_color='darkgrey'
    ),
)
demand_fig.add_trace(
    go.Bar(name='Clas IV Demand',
        x= total['Year'],
        y= (total['IV_D']), 
        marker_color='lightgrey'
    ),
)


demand_fig.update_layout(title_text="Standard Demand")
demand_fig.show()


In [523]:
demand_fig = go.Figure()
demand_fig.add_trace(
    go.Bar(name='Class I Supply',
        x= total['Year'],
        y= (total['I_SD']), 
                marker_color='grey'
    ),
)
demand_fig.add_trace(
    go.Bar(name='Class II Supply',
        x= total['Year'],
        y= (total['II_SD']), 
                marker_color='green'
    ),
)
demand_fig.add_trace(
    go.Bar(name='Class III Supply',
        x= total['Year'],
        y= (total['III_SD']), 
                marker_color='darkgrey'
    ),
)
demand_fig.add_trace(
    go.Bar(name='Clas IV Supply',
        x= total['Year'],
        y= (total['IV_SD']), 
                marker_color='lightgrey'
    ),
)
demand_fig.update_layout(title_text="Standard Supply/Demand Ratio")
demand_fig.show()

In [524]:
total['C-I Price'] = rec['Class I Price'].values
total['C-II Price'] = rec['Class II Price'].values
total['C-III Price'] = rec['Class III Price'].values
total['C-IV Price'] = rec['Class IV Price'].values
total


,Year,I_D_P,II_D_P,III_D_P,IV_D_P,I_S_P,II_S_P,III_S_P,IV_S_P,I_SD_P,...,III_S,IV_S,I_SD,II_SD,III_SD,IV_SD,C-I Price,C-II Price,C-III Price,C-IV Price
0,2022.0,669333.06,33466.653,390444.285,167333.265,1552523.251,420989.904,270000.0,255000.0,2.319508,...,270000.0,255000.0,2.171585,5.028796,0.302540,1.523905,37.25,34.00,37.50,27.75
1,2023.0,670006.98,33500.349,390837.405,167501.745,1600361.251,447861.600,263000.0,249000.0,2.388574,...,263000.0,249000.0,2.072186,5.318108,0.294400,1.486552,38.13,34.50,37.25,28.00
2,2024.0,670170.06,33508.503,390932.535,167542.515,1716248.553,469757.056,295000.0,262000.0,2.560915,...,295000.0,262000.0,2.015346,5.575514,0.330140,1.563782,36.50,34.10,36.50,28.50
3,2025.0,669075.90,33453.795,390294.275,167268.975,1774170.205,497624.000,250000.0,277000.0,2.651673,...,250000.0,277000.0,1.910215,5.942142,0.280237,1.656015,36.00,33.75,36.00,28.50
4,2026.0,669628.74,33481.437,390616.765,167407.185,1984904.963,547386.400,318000.0,266000.0,2.964187,...,318000.0,266000.0,2.050439,6.445838,0.356168,1.588940,34.00,33.00,34.00,28.50


In [525]:
total['I_%_S'] = (total['I_S_P'].values-total['I_S'].values)/total['I_S'].values
total['II_%_S'] = (total['II_S_P'].values-total['II_S'].values)/total['II_S'].values
total['III_%_S'] = (total['III_S_P'].values-total['III_S'].values)/total['III_S'].values
total['IV_%_S'] = (total['IV_S_P'].values-total['IV_S'].values)/total['IV_S'].values

total['I_%_D'] = (total['I_D_P'].values-total['I_NT_D'].values)/total['I_NT_D'].values
total['II_%_D'] = (total['II_D_P'].values-total['II_D'].values)/total['II_D'].values
total['III_%_D'] = (total['III_D_P'].values-total['III_D'].values)/total['III_D'].values
total['IV_%_D'] = (total['IV_D_P'].values-total['IV_D'].values)/total['IV_D'].values
total['III_%_D']


0   -0.5625
1   -0.5625
2   -0.5625
3   -0.5625
4   -0.5625
Name: III_%_D, dtype: float64

In [526]:
demand_fig = go.Figure()
demand_fig.add_trace(
    go.Bar(name='Class I Change in Supply',
        x= total['Year'],
        y= (total['I_%_D']),
        marker_color='lightgrey'

    ),
)
demand_fig.add_trace(
    go.Bar(name='Class II Change inSupply',
        x= total['Year'],
        y= (total['II_%_D']),
        marker_color='green'

    ),
)
demand_fig.add_trace(
    go.Bar(name='Class III Change in Supply',
        x= total['Year'],
        y= (total['III_%_D']),
        marker_color='grey'

    ),
)
demand_fig.add_trace(
    go.Bar(name='Class IV Change in Supply',
        x= total['Year'],
        y= (total['IV_%_D']),

    ),
)

demand_fig.update_layout(title_text="Percent Change in Demand")
demand_fig.show()